In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import text_encoder
from tensor2tensor.utils import registry
import json
import sklearn
from sklearn import preprocessing as skpp

C:\Users\Sayan\AppData\Local\conda\conda\envs\cs194project\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
data = pd.read_csv('./dataset/cleaned_lyrics.csv')

In [6]:
genres = data['genre'].unique()

In [13]:
data['genre_id'] = data.groupby(['genre']).ngroup()

In [14]:
data

,Unnamed: 0,index,song,year,artist,genre,lyrics,genre_id
0,0,0,ego-remix,2009,beyonce-knowles,Pop,Oh baby how you doing You know I'm gonna cut r...,8
1,1,1,then-tell-me,2009,beyonce-knowles,Pop,playin everything so easy it's like you seem s...,8
2,2,2,honesty,2009,beyonce-knowles,Pop,If you search For tenderness It isn't hard to ...,8
3,3,3,you-are-my-rock,2009,beyonce-knowles,Pop,Oh oh oh I oh oh oh I If I wrote a book about ...,8
4,4,4,black-culture,2009,beyonce-knowles,Pop,Party the people the people the party it's pop...,8
5,5,5,all-i-could-do-was-cry,2009,beyonce-knowles,Pop,I heard Church bells ringing I heard A choir s...,8
6,6,6,once-in-a-lifetime,2009,beyonce-knowles,Pop,This is just another day that I would spend Wa...,8
7,7,7,waiting,2009,beyonce-knowles,Pop,Waiting waiting waiting waiting Waiting waitin...,8
8,8,8,slow-love,2009,beyonce-knowles,Pop,I read all of the magazines while waiting aro...,8
9,9,9,why-don-t-you-love-me,2009,beyonce-knowles,Pop,N n now honey You better sit down and look aro...,8


In [34]:
mappings = data[['genre', 'genre_id']].drop_duplicates()

In [40]:
map_list = [(genre_id, genre) for genre, genre_id in mappings.values]
y = sorted(map_list)
[genre for genre_id, genre in y]

['Country',
 'Electronic',
 'Folk',
 'Hip-Hop',
 'Indie',
 'Jazz',
 'Metal',
 'Other',
 'Pop',
 'R&B',
 'Rock']

In [22]:
pd.to data[['genre_id', 'genre', 'lyrics']]

,genre_id,genre,lyrics
0,8,Pop,Oh baby how you doing You know I'm gonna cut r...
1,8,Pop,playin everything so easy it's like you seem s...
2,8,Pop,If you search For tenderness It isn't hard to ...
3,8,Pop,Oh oh oh I oh oh oh I If I wrote a book about ...
4,8,Pop,Party the people the people the party it's pop...
5,8,Pop,I heard Church bells ringing I heard A choir s...
6,8,Pop,This is just another day that I would spend Wa...
7,8,Pop,Waiting waiting waiting waiting Waiting waitin...
8,8,Pop,I read all of the magazines while waiting aro...
9,8,Pop,N n now honey You better sit down and look aro...


In [26]:
lyric_data = data['lyrics'].values
lyric_data.size

227449

In [29]:
genre_id_data = data['genre_id'].values
genre_id_data.size

227449

In [42]:
lyrics_to_id = data[['lyrics', 'genre_id']].values

In [3]:
@registry.register_problem
class LyricClassification(text_problems.Text2ClassProblem):
    """Predict lyric genre from text"""

    @property
    def approx_vocab_size(self):
        return 30000

    @property
    def is_generate_per_split(self):
        return False

    @property
    def dataset_splits(self):
        return [{
            "split": problem.DatasetSplit.TRAIN,
            "shards": 9,
        }, {
            "split": problem.DatasetSplit.EVAL,
            "shards": 1,
        }]

    @property
    def num_classes(self):
        return 11

    @property
    def class_labels(self, data_dir):
        del data_dir
        data = pd.read_csv("./dataset/genre_id_lyrics.csv")
        mappings = data[['genre', 'genre_id']].drop_duplicates()
        map_list = sorted([(genre_id, genre) for genre, genre_id in mappings.values])
        return [genre for genre_id, genre in map_list]
    
    
    def generate_samples(self, data_dir, tmp_dir, dataset_split):
        del data_dir
        del tmp_dir
        
        data = pd.read_csv('./dataset/genre_id_lyrics.csv')
        lyrics_to_id = data[['lyrics', 'genre_id']].values
        for lyric, genre_id in lyrics_to_id:
            yield {
                "inputs": lyric,
                "label": genre_id
            }

In [6]:
classification = LyricClassification()
classification.generate_data('./dataset', './tmp')

INFO:tensorflow:Generating vocab file: ./dataset\vocab.lyric_classification.30000.subwords
INFO:tensorflow:Trying min_count 500
INFO:tensorflow:Iteration 0
INFO:tensorflow:vocab_size = 24836
INFO:tensorflow:Iteration 1
INFO:tensorflow:vocab_size = 12399
INFO:tensorflow:Iteration 2
INFO:tensorflow:vocab_size = 12589
INFO:tensorflow:Iteration 3
INFO:tensorflow:vocab_size = 12580
INFO:tensorflow:Trying min_count 250
INFO:tensorflow:Iteration 0
INFO:tensorflow:vocab_size = 38480
INFO:tensorflow:Iteration 1
INFO:tensorflow:vocab_size = 18784
INFO:tensorflow:Iteration 2
INFO:tensorflow:vocab_size = 19090
INFO:tensorflow:Iteration 3
INFO:tensorflow:vocab_size = 19075
INFO:tensorflow:Trying min_count 125
INFO:tensorflow:Iteration 0
INFO:tensorflow:vocab_size = 59021
INFO:tensorflow:Iteration 1
INFO:tensorflow:vocab_size = 28459
INFO:tensorflow:Iteration 2
INFO:tensorflow:vocab_size = 28844
INFO:tensorflow:Iteration 3
INFO:tensorflow:vocab_size = 28815
INFO:tensorflow:Trying min_count 62
INFO:t